<a href="https://colab.research.google.com/github/dasparagjyoti/EEG-Channel-Selection-for-Emotion-Recognition/blob/main/Copy_of_STEP_1_Channel_Selection_(CSV_File_creation)_without_details.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive  
drive._mount('/content/drive')
!ls /content/drive/MyDrive/Deap

Mounted at /content/drive
s01.dat  s05.dat  s09.dat  s13.dat  s17.dat  s21.dat  s25.dat  s29.dat
s02.dat  s06.dat  s10.dat  s14.dat  s18.dat  s22.dat  s26.dat  s30.dat
s03.dat  s07.dat  s11.dat  s15.dat  s19.dat  s23.dat  s27.dat  s31.dat
s04.dat  s08.dat  s12.dat  s16.dat  s20.dat  s24.dat  s28.dat  s32.dat


In [ ]:
import os
import time
import pickle
import pandas as pd
import numpy as np
from scipy import signal
from scipy.signal import welch
from scipy.integrate import simps
from scipy.stats import f_oneway

from tqdm import tqdm

#!pip install scikit-learn==0.20.3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve

from scipy.signal import butter, lfilter, sosfilt, sosfreqz, freqz
from sklearn.decomposition import FastICA

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import pandas as pd
from sklearn import preprocessing
%matplotlib inline

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order = 3):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band', analog=False)
    return b, a
def butter_bandpass_filter(data, lowcut, highcut, fs, order = 5):
    b, a = butter_bandpass(lowcut, highcut, fs, order = order)
    y = lfilter(b, a, data)
    return y
def eye_movement_artifact(shyam): # parameter must be an 2D array like 32_channels*7860_data
    # Inverse that 2D array
    shyam = shyam.transpose()
    ica = FastICA()
    comps = ica.fit_transform(shyam)
    # invert the array 
    data_after = comps.transpose()
    return data_after
def signal_pro(data):
    mean_value, fs, lowcut, highcut = 0, 128, 0.5, 45
    # do the bandpass filter
    for i in range(40):
        for j in range(32):
            data[i][j] = butter_bandpass_filter(data[i][j], lowcut, highcut, fs, order=5)
    # creating dummy variable which contains same data information 
    error_eye =  np.zeros((40,32,7680))
    new_data =  np.zeros((40,32,7680))
    for i in range(40):
        for j in range(32):
            for k in range(7680):
                error_eye[i][j][k] = data[i][j][k]
                new_data[i][j][k] = data[i][j][k]
    for i in range(40):
        error_eye[i] = eye_movement_artifact(error_eye[i])
    for i in range(40):
        for j in range(32):
            mean_value = np.mean(error_eye[i][j])
            for k in range(7680):
                if(data[i][j][k] > 0.0):
                    # data is positive
                    new_data[i][j][k] = data[i][j][k] - abs(mean_value)
                else: # data is negative
                    new_data[i][j][k] = data[i][j][k] + abs(mean_value)
    return new_data

In [ ]:
def bandpower(data, sf, band, window_sec=None, relative=False):
    band = np.asarray(band)
    low, high = band
    # Define window length
    if window_sec is not None:
        nperseg = window_sec * sf
    else:
        nperseg = (2 / low) * sf
    # Compute the modified periodogram (Welch)
    freqs, psd = welch(data, sf, nperseg=nperseg)
    # Frequency resolution
    freq_res = freqs[1] - freqs[0]
    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)
    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], dx=freq_res)
    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp

In [ ]:
def get_band_power(trial, channel, band):
  bd = (0,0)
  if (band == "theta"): # drownsiness, emotional connection, intuition, creativity
    bd = (4,8)
  elif (band == "alpha"): # reflection, relaxation
    bd = (8,12)
  elif (band == "beta"): # concentration, problem solving, memory
    bd = (12,30)
  elif (band == "gamma"): # cognition, perception, learning, multi-tasking
    bd = (30,48)
  return bandpower(filter_data[trial,channel], 128, bd)

In [ ]:
def get_csf_file(subject, filter_data):
  try:
    mypath = "/content/drive/MyDrive/Sequential methods for channel selection/our code/" + subject
    os.mkdir(mypath)
  except Exception as e:
    print(e)
  #---------------------------------------- Theta ---------------------------------------
  eeg_theta = []
  for i in range (len(filter_data)):
    for j in range (len(filter_data[0])):
      eeg_theta.append(get_band_power(i,j,"theta"))
  eeg_theta = np.reshape(eeg_theta, (40, 32))
  df_theta = pd.DataFrame(data = eeg_theta, columns=eeg_channels)
  # Normalize the datset
  draft = []
  for i in range(0, len(eeg_channels)):
    draft.append(eeg_channels[i] + '_theta')
  min_max_scaler = preprocessing.MinMaxScaler()
  df_theta = min_max_scaler.fit_transform(df_theta)
  df_theta = pd.DataFrame(df_theta, columns = draft)
  df_theta.to_csv(mypath + "/" + subject + "_theta.csv", index = False, encoding = 'utf-8-sig')
  #---------------------------------------- Alpha --------------------------------------------------
  eeg_alpha = []
  for i in range (len(filter_data)):
    for j in range (len(filter_data[0])):
      eeg_alpha.append(get_band_power(i,j,"alpha"))
  eeg_alpha = np.reshape(eeg_alpha, (40, 32))

  df_alpha = pd.DataFrame(data = eeg_alpha, columns=eeg_channels)
  draft = []
  for i in range(0, len(eeg_channels)):
    draft.append(eeg_channels[i] + '_alpha')
  min_max_scaler = preprocessing.MinMaxScaler()
  df_alpha = min_max_scaler.fit_transform(df_alpha)
  df_alpha = pd.DataFrame(df_alpha, columns = draft)
  df_alpha.to_csv(mypath + "/" + subject + "_alpha.csv", index=False, encoding='utf-8-sig')
  #---------------------------------------- Beta----------------------------------------------------
  eeg_beta = []
  for i in range (len(filter_data)):
    for j in range (len(filter_data[0])):
      eeg_beta.append(get_band_power(i,j,"beta"))
  eeg_beta = np.reshape(eeg_beta, (40, 32))
  df_beta = pd.DataFrame(data = eeg_beta, columns=eeg_channels)
  draft = []
  for i in range(0, len(eeg_channels)):
    draft.append(eeg_channels[i] + '_beta')
  min_max_scaler = preprocessing.MinMaxScaler()
  df_beta = min_max_scaler.fit_transform(df_beta)
  df_beta = pd.DataFrame(df_beta, columns = draft)
  df_beta.to_csv(mypath + "/" + subject + "_beta.csv", index=False, encoding='utf-8-sig')
  #---------------------------------------- Gamma-----------------------------------------------------
  eeg_gamma = []
  for i in range (len(filter_data)):
    for j in range (len(filter_data[0])):
      eeg_gamma.append(get_band_power(i,j,"gamma"))
  eeg_gamma = np.reshape(eeg_gamma, (40, 32))
  df_gamma = pd.DataFrame(data = eeg_gamma, columns=eeg_channels)
  draft = []
  for i in range(0, len(eeg_channels)):
    draft.append(eeg_channels[i] + '_gamma')
  min_max_scaler = preprocessing.MinMaxScaler()
  df_gamma = min_max_scaler.fit_transform(df_gamma)
  df_gamma = pd.DataFrame(df_gamma, columns = draft)
  df_gamma.to_csv(mypath + "/" + subject + "_gamma.csv", index=False, encoding='utf-8-sig')
  #---------------------------------------- All Band -----------------------------------------------------
  frames = [df_theta, df_alpha, df_beta, df_gamma]
  all_bands = pd.concat(frames, axis=1)
  all_bands.shape
  all_bands.to_csv(mypath + "/" + subject + ".csv", index=False, encoding='utf-8-sig')

In [ ]:
subject_names = ["s01", "s02", "s03", "s04", "s05", "s06", "s07", "s08", "s09", "s10", "s11", "s12", "s13", "s14", "s15", "s16", "s17", "s18", "s19", "s20", "s21",
                "s22", "s23", "s24", "s25", "s26", "s27", "s28", "s29", "s30", "s31", "s32"]

In [ ]:
for subject in subject_names[1:]:
  eeg_channels = np.array(["Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1", "P3", "P7", "PO3", "O1", "Oz", "Pz", "Fp2", "AF4", "Fz", "F4", "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8", "PO4", "O2"])
  with open('/content/drive/MyDrive/Deap/' + subject + '.dat', 'rb') as f:
    raw_data = pickle.load(f, encoding = 'latin1')
  data = raw_data['data']
  reduced_eeg_data  = data[0:40, 0:32, 384:8064]
  filter_data = signal_pro(data)
  get_csf_file(subject, filter_data)